In [1]:
# Ignore SQLITE warnings related to Decimal numbers in the Chinook database
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
from pytrends.request import TrendReq
import pandas as pd
import numpy as np

In [3]:
# Login to Google. Only need to run this once, the rest of requests will use the same session.
pytrend = TrendReq()

In [4]:
# Collects the user's input for the prompt "What is your name?"
string_of_keywords = input("What interests you? ")

What interests you? tacos, sandwiches, kebabs


In [5]:
keywords = string_of_keywords.split(sep=',')
keywords 

['tacos', ' sandwiches', ' kebabs']

In [6]:
pytrend.build_payload(kw_list=keywords)

In [7]:
# Interest over time
interest_over_time_df = pytrend.interest_over_time()

In [8]:
# Interest by region
interest_by_region_df = pytrend.interest_by_region()

In [9]:
countries_df = pd.read_csv("../static/csv/countries.csv")

In [10]:
interest_over_time_df.head()

,tacos,sandwiches,kebabs,isPartial
date,,,,
2014-06-01,15,5,1,False
2014-06-08,15,5,1,False
2014-06-15,15,5,1,False
2014-06-22,16,6,1,False
2014-06-29,16,5,1,False


In [11]:
interest_by_region_df.head()

,tacos,sandwiches,kebabs
geoName,,,
Afghanistan,0,0,0
Albania,0,0,0
Algeria,0,0,0
American Samoa,0,0,0
Andorra,0,0,0


In [12]:
interest_by_region_df['total'] = interest_by_region_df.sum(axis=1)

In [13]:
interest_by_region_df.head()

,tacos,sandwiches,kebabs,total
geoName,,,,
Afghanistan,0,0,0,0
Albania,0,0,0,0
Algeria,0,0,0,0
American Samoa,0,0,0,0
Andorra,0,0,0,0


In [14]:
interest_by_region_filtered_df = interest_by_region_df[interest_by_region_df['total']>0]

In [15]:
interest_by_region_filtered_df.head()

,tacos,sandwiches,kebabs,total
geoName,,,,
Argentina,82,18,0,100
Australia,30,19,51,100
Brazil,93,4,3,100
Canada,63,35,2,100
Ecuador,86,14,0,100


In [16]:
countries_df.head()

,name,country,latitude,longitude
0,Afghanistan,AF,33.939110,67.709953
1,Albania,AL,41.153332,20.168331
2,Algeria,DZ,28.033886,1.659626
3,American Samoa,AS,-14.270972,-170.132217
4,Andorra,AD,42.546245,1.601554


In [41]:
interest_by_region_loc_df = pd.merge(interest_by_region_filtered_df, countries_df, how="inner", left_on="geoName", right_on="name")


In [42]:
interest_by_region_loc_df.head()

,tacos,sandwiches,kebabs,total,name,country,latitude,longitude
0,82,18,0,100,Argentina,AR,-38.416097,-63.616672
1,30,19,51,100,Australia,AU,-25.274398,133.775136
2,93,4,3,100,Brazil,BR,-14.235004,-51.925280
3,63,35,2,100,Canada,CA,56.130366,-106.346771
4,86,14,0,100,Ecuador,EC,-1.831239,-78.183406


In [43]:
interest_by_region_loc_df = interest_by_region_loc_df.drop(columns=['total'])

In [44]:
interest_by_region_loc_df.head()

,tacos,sandwiches,kebabs,name,country,latitude,longitude
0,82,18,0,Argentina,AR,-38.416097,-63.616672
1,30,19,51,Australia,AU,-25.274398,133.775136
2,93,4,3,Brazil,BR,-14.235004,-51.925280
3,63,35,2,Canada,CA,56.130366,-106.346771
4,86,14,0,Ecuador,EC,-1.831239,-78.183406


In [45]:
#interest_by_region_loc_df.to_sql('users', con=engine)

In [ ]:
# # DataFrame to Dictionary
# interest_by_region_loc_dict = interest_by_region_loc_df.to_dict()
# interest_over_time_dict = interest_over_time_df.to_dict()

In [ ]:
#interest_by_region_loc_dict

In [ ]:
# class Dict2Obj(object):
#     """
#     Turns a dictionary into a class
#     """
#     #----------------------------------------------------------------------
#     def __init__(self, dictionary):
#         """Constructor"""
#         for key in dictionary:
#             setattr(self, key, dictionary[key])
            
#     def to_string(self):
        

In [ ]:
# Region = Dict2Obj(interest_by_region_loc_dict)

In [ ]:
# Time = Dict2Obj(interest_over_time_dict)

In [46]:
# Import Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func, inspect, Table, Column, Integer, String, MetaData

In [47]:
db_uri = 'sqlite:///../static/db/top_trends.db'
engine = create_engine(db_uri, echo=False)
conn = engine.connect()

In [48]:
interest_by_region_loc_df.to_sql('region', con=engine, if_exists='replace')

In [49]:
interest_over_region_from_db_df = pd.read_sql_table('region',conn, index_col='index')

In [50]:
interest_over_region_from_db_df.head()

,tacos,sandwiches,kebabs,name,country,latitude,longitude
index,,,,,,,
0,82,18,0,Argentina,AR,-38.416097,-63.616672
1,30,19,51,Australia,AU,-25.274398,133.775136
2,93,4,3,Brazil,BR,-14.235004,-51.925280
3,63,35,2,Canada,CA,56.130366,-106.346771
4,86,14,0,Ecuador,EC,-1.831239,-78.183406
